In [46]:
import os
os.chdir(r'C:\Users\Navid\Documents\ds_salary_proj')

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [48]:
df = pd.read_csv('eda_data_modified')

In [50]:
# df = df.loc[df.Industry != '-1']
df = df.loc[df['Type of ownership'] != 'Unknown']

In [51]:
df = df.loc[(df.Size != '-1') & (df.Size != 'Unknown')]

In [52]:
df.columns

Index(['Unnamed: 0', 'index', 'Job Title', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'Competitors', 'per_hour',
       'avg_salary', 'FAANG', 'state', 'desc_length', 'junior', 'seniority',
       'title', 'intern', 'python', 'ML', 'big_data', 'deep_learning',
       'cloud_computing', 'graduate', 'undergrad', 'exp_year'],
      dtype='object')

In [53]:
df_final = df[['avg_salary' ,'Size','Type of ownership', 'Rating', 
               'Sector', 'Revenue','per_hour', 'seniority', 
               'state', 'cloud_computing', 'title', 'big_data', 
       'deep_learning', 'graduate', 'undergrad','exp_year']]

In [54]:
df_final = df_final[df_final.Rating != -1]

In [55]:
#removing skewness
# from scipy import stats
# df['avg_salary'] ,fitted_lambda1 = stats.boxcox(df['avg_salary'])

In [56]:
category_col =['Size','Type of ownership',  
               'Sector', 'Revenue', 'state', 'title']

In [57]:
from sklearn.preprocessing import LabelEncoder
mapping_dict={}
for col in category_col:
    le = LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    mapping_dict[col]=le_name_mapping
print(mapping_dict)

{'Size': {'1 to 50 Employees': 0, '10000+ Employees': 1, '1001 to 5000 Employees': 2, '201 to 500 Employees': 3, '5001 to 10000 Employees': 4, '501 to 1000 Employees': 5, '51 to 200 Employees': 6}, 'Type of ownership': {'College / University': 0, 'Company - Private': 1, 'Company - Public': 2, 'Contract': 3, 'Government': 4, 'Hospital': 5, 'Nonprofit Organization': 6, 'Other Organization': 7, 'Private Practice / Firm': 8, 'School / School District': 9, 'Subsidiary or Business Segment': 10}, 'Sector': {'-1': 0, 'Accounting & Legal': 1, 'Aerospace & Defense': 2, 'Agriculture & Forestry': 3, 'Arts, Entertainment & Recreation': 4, 'Biotech & Pharmaceuticals': 5, 'Business Services': 6, 'Construction, Repair & Maintenance': 7, 'Consumer Services': 8, 'Education': 9, 'Finance': 10, 'Government': 11, 'Health Care': 12, 'Information Technology': 13, 'Insurance': 14, 'Manufacturing': 15, 'Media': 16, 'Mining & Metals': 17, 'Non-Profit': 18, 'Oil, Gas, Energy & Utilities': 19, 'Real Estate': 20, 

In [58]:
# df_dummy.drop('Industry_other_industries',axis=1,inplace=True)

In [59]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df[['avg_salary', 'Rating', 'exp_year']] = scaler.fit_transform(df[['avg_salary', 'Rating', 'exp_year']])

In [61]:
from sklearn.model_selection import train_test_split
x = df_final.drop('avg_salary' , axis = 1)
y = df_final['avg_salary'].values
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 1) 

## Lets start with random grid search to narrow down the ranges

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring = 'neg_mean_absolute_error')
# Fit the random search model
rf_random.fit(X_train, y_train)




Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.1min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [37]:
print(rf_random.best_score_,rf_random.best_estimator_)

-12.485552812373891 RandomForestRegressor(bootstrap=False, max_features='sqrt', n_estimators=400)


In [64]:
# Create the parameter grid based on the results of random search 
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': range(570,650,10)
}
# Create a based model
rf = RandomForestRegressor()
grid = GridSearchCV(RandomForestRegressor(n_jobs=-1 ,random_state = 42, bootstrap = False, max_features =  'sqrt'), 
                    param_grid = param_grid, scoring = 'neg_mean_absolute_error',  cv =8).fit(X_train , y_train)

In [65]:
print(grid.best_score_,grid.best_estimator_)

-11.51880081695691 RandomForestRegressor(bootstrap=False, max_features='sqrt', n_estimators=630,
                      n_jobs=-1, random_state=42)


In [45]:
grid.score(X_test, y_test)

-9.557259655377306

In [66]:
import pickle

In [68]:
pickle.dump(grid.best_estimator_, open('model.pkl','wb'))